In [ ]:
#Prep and visualize data for TF
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.layers import Normalization
from tensorflow.keras import Sequential
from tensorflow import keras
from tensorflow.keras import layers


tf_data = data_df #pd.concat([y_df, x_df], axis=1)
train_data = tf_data.sample(frac=1.0, random_state=0)
#Use below for normal non-LOO training
test_data = tf_data.drop(train_data.index)
#use below for LOO training
test_data = train_data.drop([0]) #integer represents the index

colnames = train_data.columns
newnames = []
for name in colnames:
    arb = os.path.basename(name).split('.nii')[0]
    newnames.append(arb)
train_data.columns = newnames
train_data.tail()

snsfig = sns.pairplot(train_data)
snsfig

In [ ]:
#Describe the statistics of the data
data_description = train_data.describe().transpose()
display(data_description)

In [ ]:
#Remove the Y Variable
train_fx = train_data.copy()
test_fx = test_data.copy()

train_labels = train_fx.pop(newnames[0]) #remove the y variable
test_label = test_fx.pop(newnames[0]) #remove y vars

#Normalize the data
train_data.describe().transpose()[['mean', 'std']] #Visualize key info
normalizer = tf.keras.layers.Normalization(axis=1)
normalizer.adapt(np.array(train_fx))

first = np.array(train_fx[:1])

with np.printoptions(precision=2, suppress=True):
    print('Original:', first)
    print('Normalized:', normalizer(first).numpy())


In [ ]:
#Train the Linear Regression
linear_model = tf.keras.Sequential([normalizer, layers.Dense(units=1)])
linear_model.summary()

In [ ]:
print('Model shape below, should be shape=(# vars, 1)')
linear_model.layers[1].kernel 

In [ ]:
linear_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=.5), 
    loss='mean_absolute_error')
print('compiled')

In [ ]:
history = linear_model.fit(
    train_fx,
    train_labels,
    epochs=200,
    # Suppress logging.
    verbose=0,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.0)

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

In [ ]:
def plot_loss(history):
  train_plot = plt.figure()
  plt.plot(history.history['loss'], label='loss')
  # plt.plot(history.history['val_loss'], label='validation_loss')
  plt.ylim([0, 50])
  plt.xlabel('Epoch')
  plt.ylabel('Error [% Change ADAS]')
  plt.legend()
  plt.grid(True)
  return train_plot

train_plot = plot_loss(history)
train_plot

test_results = {}
test_results['linear_model'] = linear_model.evaluate(test_fx, test_label, verbose=0)
print('%Loss: ', test_results)

In [ ]:
#Make Predictions
try:
    test_predictions = linear_model.predict(test_fx).flatten()
    pears, p = pearsonr(test_predictions, test_label)

    test_plot = plt.figure()
    plt.axes(aspect='equal')
    plt.scatter(test_label, test_predictions)
    plt.title((f'Corr Coeff: {pears} | P Value: {p}'))
    plt.xlabel('True Vals [% Change ADAS]')
    plt.ylabel('Predicted [% Change ADAS]')
    lims = [-200, 200]
    plt.xlim(lims)
    plt.ylim(lims)
    arb = plt.plot(lims, lims)
    # yname = newnames[0]
except:
    print('Cannot make predictions')

In [ ]:
new_df = tf_data.copy()
all_y = new_df.pop(newnames[0])
all_y.tail()
all_x = new_df
all_x.tail()
overall = linear_model.predict(all_x).flatten()
pearson1, r = pearsonr(overall, all_y)
print('total corr coef: ', pearson1, ' p: ', r)


#Demosntrate on Overall Data
overall_plot = plt.figure()
plt.axes(aspect='equal')
plt.scatter(all_y, overall)
plt.title((f'Corr Coeff: {pearson1} | P Value: {r}'))
plt.xlabel('True Vals [% Change ADAS]')
plt.ylabel('Predicted [% Change ADAS]')
lims = [-210, 210]
plt.xlim(lims)
plt.ylim(lims)
arb = plt.plot(lims, lims)

In [ ]:
try:
    error = test_predictions - test_label
    error_plot = plt.figure()
    plt.hist(error, bins=25)
    plt.xlabel('Prediction Error [% Change]')
    arb = plt.ylabel('Count')
except:
    print('Cannot make error plot')

In [ ]:
#Prep Dir
if os.path.isdir(out_dir) == False:
    os.mkdir(out_dir)
out_path = os.path.join(out_dir, analysis_name)

#Save Regression Figures
try:
    train_plot.savefig((out_path+'_train_plot.png'))
except:
    print('no train plot')

try:
    test_plot.savefig((out_path+'_test_plot.png'))
except:
    print('no test plot')

try:
    overall_plot.savefig(out_path+'_overall_plot.png')
except:
    print('no overall plot')

try:
    error_plot.savefig(out_path+'_error_plot.png')
except:
    print('no error plot')

In [ ]:
#Save model for future predictions
linear_model.save(out_path+'_regression_model')

In [ ]:
#Save predictions to csv
predictions_csv = pd.DataFrame({'predictions': overall, 'actuals': all_y})
predictions_csv.tail()
predictions_csv.to_csv(out_path + '_overall_predictions.csv')

In [ ]:
# Save Pairplot and 
try:
    snsfig.savefig(out_path+'_pairplot.png')
except:
    print('no pairplot')
try:
    data_description.to_csv(out_path+'data_description.csv')
except:
    print('no data_description')